In [12]:
import os
import sys
import time
import networkx as nx
import scipy.io as sio
import scipy.sparse as sp
from scipy.spatial import distance_matrix
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.lines import Line2D
from matplotlib.animation import FuncAnimation

import pickle

%matplotlib inline

import re
numeric_const_pattern = r"""
     [-+]? 
     (?:
         (?: \d* \. \d+ )
         |
         (?: \d+ \.? )
     )
     (?: [Ee] [+-]? \d+ ) ?
     """
rx = re.compile(numeric_const_pattern, re.VERBOSE)

from IPython.display import clear_output
from IPython.display import Video

from scipy.stats.stats import pearsonr
import subprocess
from datetime import datetime


/tmp/ipykernel_3002360/1506328072.py:36: DeprecationWarning: Please import `pearsonr` from the `scipy.stats` namespace; the `scipy.stats.stats` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.stats.stats import pearsonr


In [13]:
bash_dir= "bash"
output_dir = "output"
config_file = "zinc_gps_config.csv"

df_all = pd.read_csv(os.path.join(bash_dir, config_file),index_col=0) 
df_all = df_all.reset_index(drop=True)
df_all
df=df_all
# df=df_all.iloc[[2, 16, 4, 9, 20, 24, 3, 39, 42, 41]]
# df=df_all.iloc[[2, 16, 43,44,42,1,3, 39]]
# df=df_all.iloc[[4, 9, 43,44,42,1,3, 39]]
# df=df_all.iloc[df_all.index != 6] #.iloc[[6]]
# df=df_all.iloc[df_all.index > 9] #.iloc[[6]]
# df=df_all.iloc[[0,1,2,3,4,5]]
df=df_all.iloc[[50]]
# df.loc[df["opt"]=="Lion", "lr"]=0.0001
# df.loc[df["opt"]=="Adam41", "lr"]=0.0001
# df_config.loc[df_config["opt"]=="Adam4", "lr"]=0.001
# df["lr"]=1e-4
df["lr"]=2.5e-4
df


/tmp/ipykernel_3002360/3544075960.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lr"]=2.5e-4


,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate
50,PIDAOSI,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.00025,zinc,PIDAOSI,0.0005


In [14]:
num_gpus = 1
max_threads = 36


# df["lr"]=2.5e-4

env = "cartpole"
epochs = 200
seed0 = 20
# db=-6
end_e = 0.05
exploration_fraction=0.1
decay_steps = 1
batch_size = 128
weight_decay = 0.00001
scheduler = 0 # constant learning rate no scheduled decay
buffer_size = 5000
target_network_frequency=500
# db_noise=-140
db_values = [0.0, -10.0, -15.0]


expanded_rows = []
for _, row in df.iterrows():
    if row["optimizer"] == "AdamPlus":
        for db in db_values:
            new_row = row.copy()
            new_row["db"] = db
            expanded_rows.append(new_row)
    else:
        row["db"] = 0.0
        expanded_rows.append(row)
df_expanded = pd.DataFrame(expanded_rows)
df_expanded = df_expanded.reset_index(drop=True)
df = df_expanded
df["db_noise"] = '-140.0'

In [15]:
df

,opt,beta1,beta2,seed,epoch,loss,val_mae,test_mae,lr,env,optimizer,learning_rate,db,db_noise
0,PIDAOSI,0.9,0.999,seed10,1,0.692272,0.531897,0.551542,0.00025,zinc,PIDAOSI,0.0005,0.0,-140.0


In [ ]:
running_procs = []

for i, row in df.iterrows():
    opt = row["opt"]
    beta2 = row["beta2"]
    lr = row["lr"]
    # lr = 0.0001
    db = row['db']
    db_noise = row['db_noise']
    env_copy = os.environ.copy()
    if num_gpus > 0:
        gpu_id = i % num_gpus  # round-robin assignment
        env_copy["CUDA_VISIBLE_DEVICES"] = str(gpu_id) #"1" #
    else:
        env_copy["CUDA_VISIBLE_DEVICES"] = ""
    for seed in range(seed0, seed0+10):
        outname = f"{env}_epochs{epochs}_seed{seed}_{beta2}_{opt}_{lr}.txt"
        log_path = f"{output_dir}/{outname}"

        cmd = [
            "taskset", "-c", "0-63",
            "python3.10", "dqn.py",
            f"--seed={seed}",
            f"--learning_rate={lr}",
            f"--optimizer={opt}",
            f"--beta_2={beta2}",
            f"--db={db}",
            f"--db_noise={db_noise}",
            f"--end_e={end_e}",
            f"--buffer_size={buffer_size}",
            f"--exploration_fraction={exploration_fraction}",
            f"--target_network_frequency={target_network_frequency}",
            f"--weight_decay={weight_decay}",
        ]

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"[{timestamp}][Submitting]: {' '.join(cmd)} > {log_path}")

        with open(log_path, "w") as outfile:
            proc = subprocess.Popen(
                cmd, stdout=outfile, stderr=outfile, env=env_copy, 
                # preexec_fn=os.setsid,
            )
            running_procs.append(proc)

        # Limit number of concurrent jobs
        while len(running_procs) >= max_threads:
            for p in running_procs:
                if p.poll() is not None:  # finished
                    running_procs.remove(p)
            time.sleep(30)  # wait a bit before checking again

# Wait for remaining jobs to finish
for p in running_procs:
    p.wait()


